# Carregamento e Exploração de Dados INMET

Este notebook é destinado à exploração e visualização dos dados do INMET. Para processamento completo de todos os arquivos, use o notebook `01_tratamento_dados_inmet.ipynb`.

## Acessando arquivos do INMET

In [ ]:
from pathlib import Path
import pandas as pd

# Importar a função de processamento do outro notebook
# Se estiver executando este notebook isoladamente, você pode copiar a função aqui
# ou importá-la de um módulo Python se disponível

base_path = Path("/home/jovyan/data/raw")

# Listar arquivos disponíveis
arquivos_csv = list(base_path.rglob("*.CSV"))
print(f"Total de arquivos CSV encontrados: {len(arquivos_csv)}")
arquivos_csv[:10]

[PosixPath('/home/jovyan/data/raw/2020/INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV'),
 PosixPath('/home/jovyan/data/raw/2020/INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV'),
 PosixPath('/home/jovyan/data/raw/2021/INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV'),
 PosixPath('/home/jovyan/data/raw/2021/INMET_NE_PE_A322_GARANHUNS_01-01-2021_A_31-12-2021.CSV'),
 PosixPath('/home/jovyan/data/raw/2022/INMET_NE_PE_A307_PETROLINA_01-01-2022_A_31-12-2022.CSV'),
 PosixPath('/home/jovyan/data/raw/2022/INMET_NE_PE_A322_GARANHUNS_01-01-2022_A_31-12-2022.CSV'),
 PosixPath('/home/jovyan/data/raw/2023/INMET_NE_PE_A307_PETROLINA_01-01-2023_A_31-12-2023.CSV'),
 PosixPath('/home/jovyan/data/raw/2023/INMET_NE_PE_A322_GARANHUNS_01-01-2023_A_31-12-2023.CSV'),
 PosixPath('/home/jovyan/data/raw/2024/INMET_NE_PE_A307_PETROLINA_01-01-2024_A_31-12-2024.CSV'),
 PosixPath('/home/jovyan/data/raw/2024/INMET_NE_PE_A322_GARANHUNS_01-01-2024_A_31-12-2024.CSV')]

## Função de Processamento

A função `processar_inmet()` está definida no notebook `01_tratamento_dados_inmet.ipynb`. 
Aqui vamos usá-la para processar um arquivo específico para exploração.

In [ ]:
# Definir a função de processamento (copiada do notebook de tratamento)
def processar_inmet(path: Path) -> pd.DataFrame:
    """
    Processa um arquivo CSV do INMET e retorna um DataFrame tratado.
    Esta função é a mesma usada no notebook 01_tratamento_dados_inmet.ipynb
    """
    print(f"\n📄 Lendo arquivo: {path.name}")
    
    # 1) Ler CSV bruto
    df = pd.read_csv(
        path,
        sep=';',
        skiprows=8,           # pula bloco de metadados
        encoding='latin1',
        low_memory=False
    )
    
    # 2) Renomear colunas de interesse (só as que existirem)
    mapa_renome = {
        'Data': 'data',
        'Hora UTC': 'hora',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp_ar',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
        'RADIACAO GLOBAL (Kj/m²)': 'radiacao',
        'RADIACAO GLOBAL (kJ/m²)': 'radiacao',
        'RADIACAO GLOBAL': 'radiacao',
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'vento_vel',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'vento_dir',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
    }
    df = df.rename(columns={orig: novo for orig, novo in mapa_renome.items() if orig in df.columns})

    # 3) Garante que data e hora existem
    if 'data' not in df.columns or 'hora' not in df.columns:
        raise ValueError("Colunas 'Data' e/ou 'Hora UTC' não foram encontradas após o rename.")

    # 4) Padronizar 'data' e 'hora' como string
    df['data'] = df['data'].astype(str).str.strip()
    df['hora'] = df['hora'].astype(str).str.strip()

    # 5) Remover ' UTC' da hora, se existir
    df['hora'] = df['hora'].str.replace(' UTC', '', regex=False)

    # 6) Se hora estiver como 0, 300, 1200 etc, padronizar para HH:MM
    mascara_numerica = df['hora'].str.fullmatch(r'\d{1,4}')
    df.loc[mascara_numerica, 'hora'] = (
        df.loc[mascara_numerica, 'hora']
          .str.zfill(4)
          .str[:2] + ':' + df.loc[mascara_numerica, 'hora'].str.zfill(4).str[2:]
    )

    # 7) Criar coluna datetime (deixa o pandas inferir formato)
    df['datetime'] = pd.to_datetime(
        df['data'] + ' ' + df['hora'],
        errors='coerce'
    )

    # Remover linhas onde datetime não pôde ser montado
    df = df.dropna(subset=['datetime'])
    df = df.set_index('datetime')

    # 8) Selecionar colunas de interesse (as que existirem)
    colunas_desejadas = ['hora', 'temp_ar', 'umidade', 'radiacao',
                         'vento_vel', 'precipitacao', 'pressao']
    colunas_presentes = [c for c in colunas_desejadas if c in df.columns]
    df = df[colunas_presentes]

    # 9) Converter variáveis numéricas para float
    variaveis_numericas = [c for c in colunas_presentes if c != 'hora']

    for col in variaveis_numericas:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 10) Remover colunas que estão 100% NaN (por exemplo, radiacao ausente o ano todo)
    colunas_todas_nan = [c for c in variaveis_numericas if df[c].isna().sum() == len(df)]
    if colunas_todas_nan:
        print("   ⚠ Removendo colunas 100% NaN:", colunas_todas_nan)
        df = df.drop(columns=colunas_todas_nan)
        variaveis_numericas = [c for c in variaveis_numericas if c not in colunas_todas_nan]

    # 11) Interpolar valores faltantes ao longo do tempo
    if variaveis_numericas:
        df[variaveis_numericas] = df[variaveis_numericas].interpolate(
            method='time'
        )

        # Opcional: preencher pontas (início/fim) com ffill/bfill
        df[variaveis_numericas] = df[variaveis_numericas].ffill().bfill()

    # 12) Remover apenas linhas em que TODAS as variáveis numéricas são NaN (caso sobrem)
    if variaveis_numericas:
        mask_all_nan = df[variaveis_numericas].isna().all(axis=1)
        n_all_nan = mask_all_nan.sum()
        if n_all_nan > 0:
            print(f"   ⚠ Removendo {n_all_nan} linhas 100% NaN nas variáveis numéricas.")
            df = df[~mask_all_nan]

    # 13) Criar features auxiliares
    df['hora_num'] = df.index.hour
    df['mes'] = df.index.month

    print(f"   ✅ Registros após tratamento: {len(df)}")
    return df

# Selecionar um arquivo para exploração (exemplo: Petrolina 2024)
arquivo_petrolina_2024 = list(base_path.glob("2024/*PETROLINA*.CSV"))[0]
print(f"Arquivo selecionado para exploração: {arquivo_petrolina_2024.name}")
arquivo_petrolina_2024


PosixPath('/home/jovyan/data/raw/2024/INMET_NE_PE_A307_PETROLINA_01-01-2024_A_31-12-2024.CSV')

In [ ]:
# Processar o arquivo usando a função padronizada
df = processar_inmet(arquivo_petrolina_2024)

# Visualizar os primeiros registros
df.head()


,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2024/01/01,0000 UTC,0,"970,5","970,5","969,8",NaN,"26,8","20,1","26,8","26,6","20,7",20,70.0,67.0,67.0,294.0,5,"2,2",NaN
1,2024/01/01,0100 UTC,0,"970,4","970,5","970,3",NaN,"26,4","20,7","26,8","26,3","20,7",20,71.0,66.0,71.0,256.0,"4,5","1,5",NaN
2,2024/01/01,0200 UTC,0,"970,8","970,8","970,3",NaN,"26,7","20,7","26,7","26,4","20,8","20,7",71.0,70.0,70.0,222.0,"3,4","1,4",NaN
3,2024/01/01,0300 UTC,"8,8","970,8","971,1","970,8",NaN,"23,2","21,1","26,8","23,2","21,6","20,4",89.0,69.0,88.0,335.0,"2,9",",9",NaN
4,2024/01/01,0400 UTC,"2,6","970,2","970,8","970,2",NaN,"24,3","22,1","24,7",23,"22,6","21,1",92.0,83.0,87.0,314.0,5,"2,1",NaN


In [ ]:
# Verificar colunas disponíveis após processamento
print("Colunas no DataFrame processado:")
print(df.columns.tolist())
print(f"\nTotal de colunas: {len(df.columns)}")
print(f"Total de registros: {len(df)}")

Index(['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
       'VENTO, VELOCIDADE HORARIA (m/s)', 'Unnamed: 19'],
      dtype='object')

## Informações sobre as Variáveis

Algumas variáveis presentes nos arquivos do INMET foram descartadas por não contribuírem para a identificação de padrões climáticos horários. Valores máximos/mínimos (de temperatura, umidade e orvalho) são estatísticas derivadas, não medições diretas, o que introduz ruído e distorce o comportamento real das variáveis climáticas. Da mesma forma, a rajada máxima do vento representa um pico isolado e não o padrão do vento ao longo do tempo, o que é inadequado para técnicas de agrupamento sensíveis a outliers, como K-Means. Por fim, o ponto de orvalho não é listado como obrigatório e não possui relevância direta para caracterizar a climatologia geral do período no contexto da viticultura. Por esses motivos, apenas as variáveis relevantes (temperatura, umidade, radiação, vento, precipitação e pressão) foram mantidas.

## Verificações e Estatísticas

In [ ]:
# Verificar o índice datetime
print("Período dos dados:")
print(f"  Início: {df.index.min()}")
print(f"  Fim: {df.index.max()}")
print(f"  Total de horas: {len(df)}")
print(f"\nÍndice é DatetimeIndex: {isinstance(df.index, pd.DatetimeIndex)}")


## Sobre a Criação da Coluna Datetime

A criação da coluna datetime foi necessária para consolidar as informações de data e hora em um único eixo temporal coerente. Como as estações do INMET fornecem data e hora separadas (e a hora no formato UTC sem separador), é essencial unificá-las para permitir ordenação cronológica, cálculos temporais, filtragens por horas ou períodos e construção de séries temporais. Além disso, os algoritmos de agrupamento aplicados neste projeto dependem de uma representação temporal contínua para identificar padrões climáticos. O ThingsBoard também exige timestamps unificados para publicar telemetrias. Portanto, a criação da coluna datetime é uma etapa obrigatória no tratamento dos dados.

In [ ]:
# Verificar tipos de dados
print("Tipos de dados:")
print(df.dtypes)


In [ ]:
# Verificar valores faltantes
variaveis_numericas = [c for c in df.columns if c not in ['hora']]
print("Valores faltantes por coluna:")
print(df[variaveis_numericas].isna().sum())

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
               '2024-01-01 02:00:00', '2024-01-01 03:00:00',
               '2024-01-01 04:00:00', '2024-01-01 05:00:00',
               '2024-01-01 06:00:00', '2024-01-01 07:00:00',
               '2024-01-01 08:00:00', '2024-01-01 09:00:00',
               ...
               '2024-12-31 14:00:00', '2024-12-31 15:00:00',
               '2024-12-31 16:00:00', '2024-12-31 17:00:00',
               '2024-12-31 18:00:00', '2024-12-31 19:00:00',
               '2024-12-31 20:00:00', '2024-12-31 21:00:00',
               '2024-12-31 22:00:00', '2024-12-31 23:00:00'],
              dtype='datetime64[ns]', name='datetime', length=8784, freq=None)

In [ ]:
# Estatísticas descritivas
print("Estatísticas descritivas das variáveis numéricas:")
df[variaveis_numericas].describe()

False

In [ ]:
# Visualizar amostra dos dados finais
print("Amostra dos dados processados:")
df.head(10)

datetime
2024-01-01 00:00:00    00:00
2024-01-01 01:00:00    01:00
2024-01-01 02:00:00    02:00
2024-01-01 03:00:00    03:00
2024-01-01 04:00:00    04:00
2024-01-01 05:00:00    05:00
2024-01-01 06:00:00    06:00
2024-01-01 07:00:00    07:00
2024-01-01 08:00:00    08:00
2024-01-01 09:00:00    09:00
Name: hora, dtype: object

In [ ]:
# Verificar se há coluna de radiação e seu status
if 'radiacao' in df.columns:
    n_nan_rad = df['radiacao'].isna().sum()
    print(f"Coluna 'radiacao' presente: {n_nan_rad} valores faltantes de {len(df)} ({100*n_nan_rad/len(df):.1f}%)")
else:
    print("Coluna 'radiacao' foi removida (100% de valores faltantes)")

Index(['data', 'hora', 'precipitacao', 'pressao',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)', 'temp_ar',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'umidade', 'vento_dir',
       'VENTO, RAJADA MAXIMA (m/s)', 'vento_vel', 'Unnamed: 19'],
      dtype='object')

In [ ]:
# Resumo final
print("=" * 60)
print("RESUMO DO DATASET PROCESSADO")
print("=" * 60)
print(f"Período: {df.index.min().date()} a {df.index.max().date()}")
print(f"Total de registros: {len(df):,}")
print(f"Variáveis numéricas: {len(variaveis_numericas)}")
print(f"Variáveis: {', '.join(variaveis_numericas)}")
print(f"Valores faltantes: {df[variaveis_numericas].isna().sum().sum()}")
print("=" * 60)


Index(['data', 'hora', 'precipitacao', 'pressao',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)', 'radiacao',
       'temp_ar', 'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'umidade', 'vento_dir',
       'VENTO, RAJADA MAXIMA (m/s)', 'vento_vel', 'Unnamed: 19'],
      dtype='object')

## Próximos Passos

Para processar todos os arquivos (2020-2024, Petrolina e Garanhuns) e carregar no Snowflake, execute o notebook `01_tratamento_dados_inmet.ipynb`.

In [ ]:
# Visualizar dados finais
df.head()


,hora,temp_ar,umidade,radiacao,vento_vel,precipitacao,pressao
datetime,,,,,,,
2024-01-01 00:00:00,00:00,"26,8",67.0,NaN,"2,2",0,"970,5"
2024-01-01 01:00:00,01:00,"26,4",71.0,NaN,"1,5",0,"970,4"
2024-01-01 02:00:00,02:00,"26,7",70.0,NaN,"1,4",0,"970,8"
2024-01-01 03:00:00,03:00,"23,2",88.0,NaN,",9","8,8","970,8"
2024-01-01 04:00:00,04:00,"24,3",87.0,NaN,"2,1","2,6","970,2"


## Dados Prontos para Análise

Os dados estão processados e prontos para análise. Todas as variáveis numéricas foram convertidas para float, valores faltantes foram interpolados temporalmente, e features auxiliares (hora_num, mes) foram criadas.

In [ ]:
# Verificar distribuição das variáveis (exemplo de análise exploratória)
import matplotlib.pyplot as plt

# Criar gráficos de distribuição (opcional - requer matplotlib)
# Descomente as linhas abaixo se quiser visualizar

# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# 
# for i, var in enumerate(variaveis_numericas):
#     if var in df.columns:
#         df[var].hist(ax=axes[i], bins=50)
#         axes[i].set_title(f'Distribuição de {var}')
#         axes[i].set_xlabel(var)
#         axes[i].set_ylabel('Frequência')
# 
# plt.tight_layout()
# plt.show()

print("Dados processados e prontos para análise!")


hora             object
temp_ar         float64
umidade         float64
radiacao        float64
vento_vel       float64
precipitacao    float64
pressao         float64
dtype: object

## Exportar Dados (Opcional)

Se quiser salvar este dataset processado localmente:

In [ ]:
# Exemplo: salvar o dataset processado (descomente para usar)
# caminho_saida = Path("/home/jovyan/data/processed/petrolina_2024_exploratorio.csv")
# df.to_csv(caminho_saida)
# print(f"Dataset salvo em: {caminho_saida}")

print("Para processar todos os arquivos, use o notebook 01_tratamento_dados_inmet.ipynb")

temp_ar         5301
umidade         5305
radiacao        8784
vento_vel       5301
precipitacao    5374
pressao         5301
dtype: int64

## Nota sobre Interpolação

Os dados foram processados usando interpolação temporal para preencher valores faltantes. Isso mantém a continuidade das séries temporais e é uma prática recomendada para dados meteorológicos. Para mais detalhes, consulte o notebook `01_tratamento_dados_inmet.ipynb`.

3406

temp_ar         0
umidade         0
vento_vel       0
precipitacao    0
pressao         0
dtype: int64

,hora,temp_ar,umidade,vento_vel,precipitacao,pressao
datetime,,,,,,
2024-01-01 00:00:00,00:00,26.8,67.0,2.2,0.0,970.5
2024-01-01 01:00:00,01:00,26.4,71.0,1.5,0.0,970.4
2024-01-01 02:00:00,02:00,26.7,70.0,1.4,0.0,970.8
2024-01-01 03:00:00,03:00,23.2,88.0,0.9,8.8,970.8
2024-01-01 04:00:00,04:00,24.3,87.0,2.1,2.6,970.2
